# Worksheet 13

Name:  Sangjoon Lee
UID: U79516048

### Topics

- Support Vector Machines
- Recommender Systems

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

# Dataset
X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

# Initializing w and b
w = np.array([1, 1])
b = 0

# Perceptron Parameters
epochs = 100
alpha = .05
expanding_rate = .99
retracting_rate = 1.1

def snap(x, w, b, error):
    """
        Plot the street induced by w and b.
        Circle the point x in red if it was
        misclassified or in yellow if it was
        classified correctly.
    """

    xplot = np.linspace(-3, 3)
    cs = np.array([x for x in 'gb'])

    svm = - (w[0] / w[1]) * xplot - (b / w[1])
    left_svm = - (w[0] / w[1]) * xplot - (b / w[1]) - 1 / w[1]
    right_svm = - (w[0] / w[1]) * xplot - (b / w[1]) + 1 / w[1]

    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)
    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='y',fill=False))
    ax.plot(xplot, left_svm, 'r--', lw=2)
    ax.plot(xplot, svm, 'r-', lw=2)
    ax.plot(xplot, right_svm, 'r--', lw=2)
    ax.set_xlim(min(X[:, 0]) - 1, max(X[:,0]) + 1)
    ax.set_ylim(min(X[:, 1]) - 1, max(X[:,1]) + 1)
    fig.savefig(TEMPFILE)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def accuracy(w, b):
    acc = 0
    for x in zip(X, Y):
        ypred = np.dot(w, x) + b
        if (ypred >= 0 and y > 0) or (ypred < 0 and y < 0):
            acc += 1
    return acc / len(X)

images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    x, y = X[i], Y[i]
    error = False
    ypred = np.dot(w, x) + b
    if (ypred >= 0 and y > 0) or (ypred < 0 and y < 0):
        # correctly classified
        if ypred <= 1 and ypred >= -1:
            # but the point is in the street
            w = w + alpha * y * x * retracting_rate
            b += alpha * y * retracting_rate
        else:
            # the point is in the margin
            w = w * expanding_rate
            b *= expanding_rate
    else:
        # misclassified
        w = w + alpha * y * x * expanding_rate
        b += alpha * y * expanding_rate
        
    images.append(snap(x, w, b, error))
    # print("training accuracy = ", accuracy(w, b))

images[0].save(
    'svm.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [12]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = .05
expanding_rate = .99
retracting_rate = 1.1

X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

alpha_i = np.zeros((len(X),))
b = 0

def snap(x, alpha_i, b, error):
    # create a mesh to plot in
    h = .01  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    meshData = np.c_[xx.ravel(), yy.ravel()]
    cs = np.array([x for x in 'gb'])
    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)

    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='y',fill=False))
   
    Z = predict_many(alpha_i, b, meshData)
    Z = np.array([0 if z <=0 else 1 for z in Z]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.5, cmap=plt.cm.Paired)
    fig.savefig(TEMPFILE)
    plt.close()
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def predict_many(alpha_i, b, Z):
    res = []
    for i in range(len(Z)):
        res.append(predict(alpha_i, b, Z[i]))
    return np.array(res)

def predict(alpha_i, b, x):
    wx = 0
    for i in range(len(X)):
        wx += alpha_i[i] * polynomial(X[i], x, 0, 2)
    return wx + b

def accuracy(w, b):
    acc = 0
    for x in zip(X, y):
        ypred = np.dot(w, x) + b
        if (ypred >= 0 and y > 0) or (ypred < 0 and y < 0):
            acc += 1
    return acc / len(X)

images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    error = False
    x, y = X[i], Y[i]
    ypred = predict(alpha_i, b, x)
    if (ypred >= 0 and y > 0) or (ypred < 0 and y < 0):
        # correctly classified
        if ypred <= 1 and ypred >= -1:
            # but the point is in the street
            alpha_i[i] += learning_rate * y
            b += learning_rate * y * retracting_rate
            alpha_i = alpha_i * retracting_rate
        else:
            # the point is in the margin
            alpha_i = alpha_i * expanding_rate
            b *= expanding_rate
    else:
        # misclassified
        alpha_i[i] += learning_rate * y
        b += learning_rate * y * expanding_rate
        alpha_i = alpha_i * expanding_rate

    images.append(snap(x, alpha_i, b, error))

images[0].save(
    'svm_dual.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [11]:
def polynomial(x_i, x_j, c, n):
    return (np.dot(x_i, x_j) + c) ** n

## Recommender Systems

In the example in class of recommending movies to users we used the movie rating as a measure of similarity between users and movies and thus the predicted rating for a user is a proxy for how highly a movie should be recommended. So the higher the predicted rating for a user, the higher a recommendation it would be.

a) Consider a streaming platform that only has "like" or "dislike" (no 1-5 rating). Describe how you would build a recommender system in this case.

Since it's only true/false value here, the recommender system should be simplified to recommending a movie if a user has enough likes in certain types of movies, rather than a threshold of values for whether it should be recommended it or not.

b) Describe 3 challenges of building a recommender system

Lack of data: The system becomes effective with lots of data to begin with. If there's not enough data to be trained with, the accuracy of the recommender system will be low.

Changing data/user preferences: The trends are always changing, and so are the user preferences, so it's hard to be accurate with recommendations when the user's preference or general trend shifts in a direction that the system is not trained upon.

Unpredicatability: There may be some items where there's a distinct division between those who like/prefer the item and those who don't, regardless of their general user trend. These items will be hard to recommend for the system as these count as outliers in terms of data analysis.

c) Why is SVD not an option for collaborative filtering?

There is little to no explanation as to why an item is recommended to an user.